# Disentangled Causal Effect Variational Autoencoder

**Inputs:**
- data/heart_disease_cleaned.csv

**Outputs:**
- DCEVEA model
- data/fair_disease_dcevae.csv
- data/cf_disease_dcevea.csv

## Setup and imports

In [1]:
try:
  from google.colab import userdata
  from google.colab import drive
  drive.mount('/content/drive')
  PROJECT_ROOT = userdata.get('PROJECT_ROOT')
except ImportError:
  PROJECT_ROOT = '/'


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as utils
import re
from torch import nn

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Classes and functions





### Utils

In [93]:
def make_loader(X_ind, X_desc, X_corr, X_sens, Y, index, batch_size=32):
  X_ind_fact = X_ind[index]
  X_desc_fact = X_desc[index]
  X_corr_fact = X_corr[index]
  X_sens_fact = X_sens[index]
  Y_fact = Y[index]

  # Permuted set for the discriminator
  permuted_indices = np.random.permutation(X_ind_fact.shape[0])
  X_ind_fake = X_ind[permuted_indices]
  X_desc_fake = X_desc[permuted_indices]
  X_corr_fake = X_corr[permuted_indices]
  X_sens_fake = X_sens[permuted_indices]
  Y_fake = Y[permuted_indices]

  X_ind_tensor = torch.tensor(X_ind_fact, dtype=torch.float32)
  X_desc_tensor = torch.tensor(X_desc_fact, dtype=torch.float32)
  X_corr_tensor = torch.tensor(X_corr_fact, dtype=torch.float32)
  X_sens_tensor = torch.tensor(X_sens_fact, dtype=torch.float32)
  Y_tensor = torch.tensor(Y_fact, dtype=torch.float32)
  X_ind_tensor_2 = torch.tensor(X_ind_fake, dtype=torch.float32)
  X_desc_tensor_2 = torch.tensor(X_desc_fake, dtype=torch.float32)
  X_corr_tensor_2 = torch.tensor(X_corr_fake, dtype=torch.float32)
  X_sens_tensor_2 = torch.tensor(X_sens_fake, dtype=torch.float32)
  Y_tensor_2 = torch.tensor(Y_fake, dtype=torch.float32)

  dataset = utils.TensorDataset(X_ind_tensor, X_desc_tensor, X_corr_tensor, X_sens_tensor, Y_tensor,
                                X_ind_tensor_2, X_desc_tensor_2, X_corr_tensor_2, X_sens_tensor_2, Y_tensor_2)
  loader = utils.DataLoader(dataset, batch_size=batch_size, shuffle=True)

  return loader

def make_bucketed_loader(dataset, map, val_size=0.1, test_size=0.1, batch_size=32, seed=4):
  '''
    Creates train, validation and test DataLoader for the given dataset, \
    separating features into independent, sensitive, descendant, and correlated features.

    Input:
      - dataset: a pandas DataFrame
      - map: a dictionary mapping feature names to buckets
      - val_size: the proportion of the dataset to use for validation
      - test_size: the proportion of the dataset to use for testing
      - batch_size: the batch size for the DataLoader
      - seed: a seed for the random number generator

    Output:
      - train_loader: Training DataLoader
      - val_loader: Validation DataLoader
      - test_loader: Testing DataLoader
  '''
  np.random.seed(seed=seed)

  ## BUCKET DATASET
  # Independent, Descendant, Correlated features
  r_ind = re.compile(f'{"|".join(map['ind'])}')
  X_ind = dataset.filter(regex=r_ind).to_numpy()
  r_desc = re.compile(f'{"|".join(map['desc'])}')
  X_desc = dataset.filter(regex=r_desc).to_numpy()
  r_corr = re.compile(f'{"|".join(map['corr'])}')
  X_corr = dataset.filter(regex=r_corr).to_numpy()

  # Sensitive attribute and Target
  X_sens = dataset[map['sens']].to_numpy().reshape(-1, 1)
  Y = dataset[map['target']].to_numpy().reshape(-1, 1)

  ## TRAIN-VAL-TRAIN SPLIT
  N = X_ind.shape[0]
  shuffled_indices = np.random.permutation(N)
  val_count = int(N * val_size)
  test_count = int(N * test_size)
  val_index = shuffled_indices[:val_count]
  test_index = shuffled_indices[val_count:val_count+test_count]
  train_index = shuffled_indices[val_count+test_count:]

  # Training loader
  train_loader = make_loader(X_ind, X_desc, X_corr, X_sens, Y, train_index, batch_size)

  # Validation loader
  val_loader = make_loader(X_ind, X_desc, X_corr, X_sens, Y, val_index, batch_size)

  # Test loader
  test_loader = make_loader(X_ind, X_desc, X_corr, X_sens, Y, test_index, batch_size)

  return train_loader, val_loader, test_loader

### DCEVAE Model

In [ ]:
class DCEVAE(nn.Module):
  def __init__(self, ind_dim, corr_dim, desc_dim, sens_dim, target_dim, args):
    super(DCEVAE, self).__init__()
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    self.ind_dim = ind_dim
    self.corr_dim = corr_dim
    self.desc_dim = desc_dim
    self.sens_dim = sens_dim
    self.target_dim = target_dim
    self.args = args
    self.device = args.device
    self.uc_dim = args.uc_dim
    self.ud_dim = args.ud_dim
    self.u_dim = self.uc_dim + self.ud_dim
    self.h_dim = args.h_dim

    # Activation function
    if (args.act_fn == 'relu'):
      self.act_fn = nn.LeakyReLU()
    elif (args.act_fn == 'tanh'):
      self.act_fn = nn.Tanh()

    # Encoder (X_ind, X_corr, X_sens, Y) -> U_c
    input_dim = ind_dim + corr_dim + sens_dim + target_dim
    self.encoder_corr = nn.Sequential(
        nn.Linear(input_dim, self.h_dim),
        self.act_fn,
        nn.Linear(self.h_dim, self.h_dim),
        self.act_fn
    )
    self.mu_corr = nn.Linear(self.h_dim, self.uc_dim)
    self.logvar_corr = nn.Linear(self.h_dim, self.uc_dim)

    # Encoder (X_ind, X_desc, X_sens, Y) -> U_d
    input_dim = ind_dim + desc_dim + sens_dim + target_dim
    self.encoder_desc = nn.Sequential(
        nn.Linear(input_dim, self.h_dim),
        self.act_fn,
        nn.Linear(self.h_dim, self.h_dim),
        self.act_fn
    )
    self.mu_desc = nn.Linear(self.h_dim, self.ud_dim)
    self.logvar_desc = nn.Linear(self.h_dim, self.ud_dim)

    # Decoder (U_c, X_ind) -> X_corr
    self.decoder_corr = nn.Sequential(
        nn.Linear(self.uc_dim + ind_dim, self.h_dim),
        self.act_fn,
        nn.Linear(self.h_dim, self.corr_dim)
    )

    # Decoder (U_desc, X_sens, X_ind) -> X_desc
    self.decoder_desc = nn.Sequential(
        nn.Linear(self.ud_dim + sens_dim + ind_dim, self.h_dim),
        self.act_fn,
        nn.Linear(self.h_dim, self.desc_dim)
    )

    # Decoder (U_corr, U_desc, X_ind, X_sens) -> Y
    self.decoder_target = nn.Sequential(
        nn.Linear(self.u_dim + ind_dim + sens_dim, self.h_dim),
        self.act_fn,
        nn.Linear(self.h_dim, self.target_dim)
    )

    # Discriminator
    self.discriminator = nn.Sequential(
        nn.Linear(self.u_dim + sens_dim, self.h_dim),
        nn.LeakyReLU(0.2, True),
        nn.Linear(self.h_dim, self.h_dim),
        nn.LeakyReLU(0.2, True),
        nn.Linear(self.h_dim, self.h_dim),
        nn.LeakyReLU(0.2, True),
        nn.Linear(self.h_dim, 2)
    )

    self.init_params()

  def init_params(self):
    main_act_fn = "tanh" if isinstance(self.act_fn, nn.Tanh) else "relu"

    for name, module in self.named_modules():
      if isinstance(module, nn.Linear):
        if "discriminator" in name:
          # Kaiming normal for the LeakyReLU in the Discriminator
          nn.init.kaiming_normal(module.weight, a=0.2, nonlinearity="leaky_relu")
        else:
          # Xavier normal for Tanh, Kaiming normal for LeakyReLU in the encoders/decoders
          if main_act_fn == "tanh":
            nn.init.xavier_normal(module.weight)
          else:
            nn.init.kaiming_normal(module.weight, a=0.01, nonlinearity="leaky_relu")

        if module.bias is not None:
          nn.init.constant_(module.bias, 0)



## Data preparation

In [94]:
heart_disease = pd.read_csv(PROJECT_ROOT + '/data/heart_disease_cleaned.csv')

# Hot-on encoding for categorical features
heart_disease_encoded = pd.get_dummies(heart_disease, columns=['cp','ecg','slope'], drop_first=True, dtype=int)

feature_mapping = {
    'ind': ['age'], # Features independent of the protected attribute and unconfounded
    'sens': 'sex', # Sensitive attribute
    'desc': ['cp', 'ecg', 'ang'], # Features descendant of the protected attribute
    'corr': ['bp', 'chol', 'fbs', 'mhr', 'st', 'slope'], # Features correlated with the protected attribute
    'target': 'cvd' # Target outcome
}

# Bucketed data loaders for training , validation, and test
train_loader, val_loader, test_loader = make_bucketed_loader(heart_disease_encoded, feature_mapping)